# Section 6

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.size": 16,
    'text.latex.preamble': r"\usepackage{amsmath}"
})

In [2]:
import irbasis3

## Generate IR basis by singular value expansion

In [3]:
# Create kenerl for lambda_ = 100 and beta = 10
# The default value of eps ``cutoff for singular values" is
# set to 2.2e-16.
# In this sample, we compute fewer basis functions.
lambda_ = 100
beta = 10
wmax = lambda_/beta
K = irbasis3.KernelFFlat(lambda_=100)
basis = irbasis3.FiniteTempBasis(
    K, statistics='F', beta=beta, eps=1e-10)

## Singular values

In [4]:
basis.s

array([1.45387261e+00, 1.24137141e+00, 7.98387159e-01, 5.34115485e-01,
       3.19202855e-01, 1.88195405e-01, 1.06269952e-01, 5.84664007e-02,
       3.12626140e-02, 1.62959998e-02, 8.28608690e-03, 4.11383766e-03,
       1.99543191e-03, 9.46115126e-04, 4.38678206e-04, 1.98971153e-04,
       8.83076718e-05, 3.83596425e-05, 1.63120097e-05, 6.79164595e-06,
       2.76915058e-06, 1.10582146e-06, 4.32562293e-07, 1.65764896e-07,
       6.22400190e-08, 2.28998146e-08, 8.25716208e-09, 2.91821977e-09,
       1.01098806e-09, 3.43374895e-10])

In [ ]:
plt.semilogy(basis.s, marker='o', ls='')
plt.xlabel(r'$l$')
plt.ylabel(r'$S_l$')
plt.tight_layout()
plt.savefig('ir_basis_svals.pdf', transparent=True)

## $U_l(\tau)$ and $V_l(\omega)$

In [ ]:
fig = plt.figure(figsize=(6,8))
ax1 = plt.subplot(211)
ax2 = plt.subplot(212)
axes = [ax1, ax2]

taus = np.linspace(0, beta, 1000)
omegas = np.linspace(-wmax, wmax, 1000)

for l in range(4):
    ax1.plot(taus, basis.u[l](taus), label=f'$l={l}$')
    ax2.plot(omegas, basis.v[l](omegas), label=f'$l={l}$')

ax1.set_xlabel(r'$\tau$')
ax2.set_xlabel(r'$\omega$')
ax1.set_ylabel(r'$U_l(\tau)$')
ax2.set_ylabel(r'$V_l(\omega)$')
ax1.set_xlim([0,beta])
ax2.set_xlim([-wmax, wmax])

ax1.legend(loc='best', frameon=False)
ax2.legend(loc='best', frameon=False)
plt.tight_layout()
plt.savefig('ir_basis_functions.pdf', transparent=True)

In [ ]:
# Three Gaussian peaks (normalized to 1)
gaussian = lambda x, mu, sigma: np.exp(-((x-mu)/sigma)**2)/(np.sqrt(np.pi)*sigma)

rho = lambda omega: 0.2*gaussian(omega, 0.0, 0.15) + \
    0.4*gaussian(omega, 1.0, 0.8) + 0.4*gaussian(omega, -1.0, 0.8)

omegas = np.linspace(-5, 5, 1000)
plt.xlabel(r'$\omega$')
plt.ylabel(r'$\rho(\omega)$')
plt.plot(omegas, rho(omegas))

In [ ]:
# Set up Composite Gauss-Legendre quadrature
# Let -wmax = w_1 < w_2 < ... < w_{N-1} < w_N = wmax be a partition of [-wmax, wmax],
# where w_2, ..., w_{N-1} are the roots of the highest basis function in omega.
# We apply Gauss-Legendre quadrature to each integral. 
from numpy.polynomial.legendre import leggauss

def scale_quad(x, w, xmax, xmin):
    """ Scale weights and notes of quadrature to the interval [xmin, xmax] """
    assert xmin < xmax
    dx = xmax - xmin
    w_ = 0.5 * dx * w
    x_ = (0.5 * dx) * (x + 1) + xmin
    return x_, w_

# Roots of highest V_l
roots = basis.v[-1].roots()
bins = np.hstack((-wmax, roots, wmax))

x, w = leggauss(deg=20)
omegas_quad = []
weights_quad = []
for i in range(bins.size-1):
    x_, w_ = scale_quad(x, w, bins[i+1], bins[i])
    omegas_quad.append(x_)
    weights_quad.append(w_)
omegas_quad = np.hstack(omegas_quad)
weights_quad = np.hstack(weights_quad)

In [ ]:
# Check orthogonality condition
for i in range(basis.size):
    for j in range(i, basis.size):
        print(i, j, weights_quad @ (basis.v[j](omegas_quad) * basis.v[i](omegas_quad)))